In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from tqdm import tqdm
import pandas as pd
import datajoint as dj

# get data source

In [2]:
%%capture
nda = dj.create_virtual_module('microns_phase3_nda','microns_phase3_nda')
radtune = dj.create_virtual_module('pipeline_radtune', 'pipeline_radtune')

# get tuned responses

In [3]:
def get_trial_avg(unit_key, inter_trial_begin=(15/16-.5)/2, inter_trial_end=(15/16-.5)/2):
    '''
    return sorted directions and trial mean responses
    '''
    trace, frame_times, ms_delay = ((nda.Activity & unit_key) * nda.ScanTimes * nda.ScanUnit).fetch1('trace', 'frame_times', 'ms_delay')
    f2a = interp1d(frame_times+ms_delay/1000,trace)  # interpolate from frame times to activity

    trials = pd.DataFrame(
        (nda.Trial * nda.Monet2 & unit_key).fetch('onsets','directions', 'start_frame_time', 'end_frame_time',as_dict=True)
    )

    trials['dir_time_bin'] = trials.apply(
        axis=1, 
        func=lambda trial : np.append((np.array(trial.onsets).ravel() + trial.start_frame_time), trial.end_frame_time)
    )  # add start time to onsets and append the end frame time
    trials['directions'] = trials.directions.apply(lambda d : np.array(d).ravel())

    ca_frame_hz = 6.3

    # QC: make sure all the resampled directions have the same amount of bins
    qc_trial_time_diff = trials.dir_time_bin.apply(
        func={
            'length_diff_range':lambda t : np.ptp(np.diff(t)),
            'min_length':lambda t : np.min(np.diff(t)),
            'max_length':lambda t : np.max(np.diff(t)),
        }, 
    ).to_frame().reset_index(names=['stats', 'trial']).pivot(columns='stats', values='dir_time_bin', index='trial')
    qc_trial_time_diff = qc_trial_time_diff.assign(
        min_bins=((qc_trial_time_diff.min_length - inter_trial_begin - inter_trial_end) * ca_frame_hz).astype(int),
        max_bins=((qc_trial_time_diff.max_length - inter_trial_begin - inter_trial_end) * ca_frame_hz).astype(int),
    )
    if len(np.unique(np.c_[qc_trial_time_diff.min_bins.to_numpy(), qc_trial_time_diff.max_bins.to_numpy()])) != 1:
        print('QC: subtrial length differences')
        display(qc_trial_time_diff.head())
        raise ValueError('trial length variation led to difference in number of resampled points!')
    n_resampled_bins = int(qc_trial_time_diff.min_bins.unique()) + 1


    # resample responses at ca imaging frame rate
    sorted_dirs = np.sort((np.unique(np.stack(trials.directions.to_numpy()))))
    responses = np.full([len(trials), len(sorted_dirs), n_resampled_bins], np.nan)  # trial x dir x time bin
    for i, trial in enumerate(trials.itertuples()):
        assert (sorted_dirs == np.sort(trial.directions)).all()
        for _dir, start, end in zip(trial.directions, trial.dir_time_bin[:-1], trial.dir_time_bin[1:]):
            j = (sorted_dirs == _dir).nonzero()[0][0]
            sampling_time = np.arange(start + inter_trial_begin, end - inter_trial_end, 1 / ca_frame_hz)
            assert len(sampling_time) == n_resampled_bins
            responses[i, j] = f2a(sampling_time)
    assert not np.isnan(responses).any()
    responses = responses.mean(axis=0)  # trial average
    return sorted_dirs, responses

In [4]:
key_limit = 5
inter_trial_begin = (15/16-.5)/2 # time to skip at beginning of subtrial (sec)
inter_trial_end = (15/16-.5)/2 # time to skip at the end of subtrials (sec)
t_edges = np.linspace(0,2*np.pi,17)-np.pi/16
von_keys = []
for i in range(16):
    bin_center = (((t_edges[i+1] - t_edges[i]) / 2) + t_edges[i])
    von_rest = f'pref_theta > {t_edges[i]} and pref_theta < {t_edges[i+1]}'
    von_rel = (radtune.VonFit.Unit & {'animal_id':17797, 'vonfit_method':3, 'ori_type':"dir"} & 
               nda.ScanInclude() & von_rest)
    pref_thetas, keys = von_rel.fetch('pref_theta', 'KEY', order_by='von_pred_adv DESC',limit=key_limit)
    [k.update({'pref_theta': p * 180/np.pi, 'bin': bin_center * 180/np.pi}) for p,k in zip(pref_thetas, keys)];
    von_keys.extend(keys)
von_keys = pd.DataFrame(von_keys).sort_values('pref_theta')

# get trial average responses
sorted_dirs = np.sort((nda.Trial * nda.Monet2 & von_keys).fetch('directions', limit=1)[0])
responses = []  # list of dir x time bin arrays
for _, unit_key in tqdm(von_keys.iterrows()):
    _dirs, _responses = get_trial_avg(dict(unit_key), inter_trial_begin, inter_trial_end)
    assert (_dirs == sorted_dirs).all()
    responses.append(_responses)

responses = np.stack(responses, axis=0)  # neuron x dir x time bin
responses = np.reshape(responses, [len(responses), -1])  # neuron x time bin
# independently normalize each neuron by the min and max
responses -= np.min(responses,axis=1,keepdims=True)
responses /= np.max(responses,axis=1,keepdims=True)

80it [00:09,  8.24it/s]


# save data

In [5]:
np.save('directions.npy', sorted_dirs)
von_keys.to_pickle('tuned_units.pkl')
np.save('tuned_unit_responses.npy', responses)